In [56]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Price of the car.

In [57]:
maxProductPrice = 400000

Information about customers

In [58]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}

Methods to help compute the market share for each customer type (based on logit model).

In [59]:
def getTotalUtilityForCustomer(car : COTSCar, c : str):
    total_utility = 0
    perf_utility = car.partworth_objectives(weights=cWts[c])[0]
    total_utility += (1-cPriceFocus[c])*perf_utility - cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar]): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(competitorCar, c) for competitorCar in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

Create a set of competitors.

In [60]:
profitMargin = 0.1
competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitorCar.price = (1+profitMargin)*competitorCar.cost()
    competitorCar.name = "Competitor Car " + str(i)
    competitors.append(competitorCar)

names = [car.name for car in competitors]
designs = [car.vector for car in competitors]
salesPrices = ['$' + format(car.price, ",.2f") for car in competitors]
costs = ['$' + format(car.cost(), ",.2f") for car in competitors]
competitorsDF = pd.DataFrame({
    'Name': names,
    'Design': designs,
    'SalesPrice': salesPrices,
    'ProductCost': costs
})
competitorsDF

,Name,Design,SalesPrice,ProductCost
0,Competitor Car 0,"[1, 0, 4, 7, 7, 0, 3, 5, 8, 3, 57, 106, 0, 2, ...","$5,055.17","$4,595.61"
1,Competitor Car 1,"[2, 6, 0, 9, 0, 0, 4, 15, 23, 1, 55, 98, 1, 2,...","$47,641.19","$43,310.17"
2,Competitor Car 2,"[5, 8, 6, 7, 4, 6, 1, 0, 13, 1, 28, 20, 1, 3, ...","$3,729.43","$3,390.39"


Create a random car and compute market share

In [61]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
product.price = (1+profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

utilities = [getTotalUtilityForCustomer(product, c) for c in customers]
market_shares = [getMarketShare(product, c, competitors) for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors)*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})

#market_shares_df = pd.DataFrame(list(market_shares.items()), columns=['Customer Type', 'MarketShare'])
#customer_demand_df = pd.DataFrame(list(expected_customer_demand.items()), columns=['Customer Type', 'Expected Demand'])
#df = pd.merge(market_shares_df, customer_demand_df, on='Customer Type')
df

OUR CAR:
Design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
Sales price: $ 7,291.86
Cost: $ 6,628.96


,Name,Utility for our car,Market share,Expected demand
0,CustomerType1,-0.243419,0.208389,43.136514
1,CustomerType2,-0.883626,0.122253,44.255600
2,CustomerType3,-0.992310,0.181468,10.162198
3,CustomerType4,-0.414773,0.158928,44.817697
4,CustomerType5,-0.612287,0.130981,81.470254
5,CustomerType6,-0.389859,0.129445,46.988417
6,CustomerType7,-1.096869,0.118462,116.684915
7,CustomerType8,-0.488083,0.178894,137.032508
8,CustomerType9,-0.920556,0.149877,17.385781
9,CustomerType10,-0.666616,0.256911,196.022966
